In [1]:
import requests
import time
from json import JSONDecodeError

def request_get_errorprone(*args, **kwargs):
    status_code = 500
    counter = 0
    while status_code != 200 and counter < 10:
        R = requests.get(*args, **kwargs)
        status_code = R.status_code
        if status_code == 200:
            return R
        else:
            time.sleep(0.5)
            counter += 1
            print("Request attempt {} failed".format(counter))
    print("Something is wrong with this request, made 10 times, all errors")
    print(R.url)
    return R
            

class OmekaSite(object):
    """Class provides an interface for interacting with an Omeka site"""
    def __init__(self, url, api_key):
        url = url.strip('/')
        if url[-3:] != "api":
            url += "/api"
        self.url = url
        self.api_key = api_key
    def get_element_set(self, name):
        try:
            sets_request = request_get_errorprone(self.url+"/element_sets")
            element_sets = sets_request.json()
        except JSONDecodeError:
            return sets_request
        for es in element_sets:
            if es['name'] == name:
                elements_request = request_get_errorprone(self.url+"/elements",params={'element_set':es['id']})
                try:
                    elements = elements_request.json()
                except JSONDecodeError:
                    return elements_request
                output = {}
                for e in elements:
                    output[e['name'].lower()] = e
                return output
        return None
    def make_element_set_lookup(self, name):
        element_set = self.get_element_set(name)
        lookup = {}
        for k,v in element_set.items():
            key = "{}::{}".format(name,v['name'])
            value = v['id']
            lookup[key] = value
        return lookup
    def item_element(self,element_id,text,html=False):
        element = {"text":text,"html":html,"element":{"id":element_id}}
        return element
    def make_item(self,element_list,public=False,featured=False):
        item = {"public":public, "featured":featured, "element_texts":element_list}
        return item
    def item_from_librarycloud(self, lc_result):
        dc_lookup = self.make_element_set_lookup("Dublin Core")
        iiif_lookup = self.make_element_set_lookup("IIIF Item Metadata")
        print(iiif_lookup)
        item = {"public":False,"featured":False,"element_texts":[]}
        for k,v in lc_result.dc_data.items():
            lookup_key = "Dublin Core::{}".format(k.capitalize())
            element_id = dc_lookup[lookup_key]
            if type(v) == list:
                for text in v:
                    item['element_texts'].append(self.item_element(element_id,text))
            else:
                item['element_texts'].append(self.item_element(element_id,v))
        canvas_element = self.item_element(iiif_lookup['IIIF Item Metadata::JSON Data'],lc_result.canvas)
        item['element_texts'].append(canvas_element)
        return item
    def post_item(self,item):
        endpoint = self.url + "/items"
        R = requests.post(endpoint,json=item,params={'key':self.api_key})
        try:
            return R.json()
        except:
            return R

In [2]:
site = OmekaSite("https://cb51.omeka.fas.harvard.edu","630d3045363b980294b3fed669f37c67d785bb9b")

In [3]:
lookup = site.make_element_set_lookup("Dublin Core")

In [4]:
lookup.update(site.make_element_set_lookup("IIIF Item Metadata"))
lookup.update(site.make_element_set_lookup("IIIF File Metadata"))

In [5]:
lookup

{'Dublin Core::Contributor': 37,
 'Dublin Core::Coverage': 38,
 'Dublin Core::Creator': 39,
 'Dublin Core::Date': 40,
 'Dublin Core::Description': 41,
 'Dublin Core::Format': 42,
 'Dublin Core::Identifier': 43,
 'Dublin Core::Language': 44,
 'Dublin Core::Publisher': 45,
 'Dublin Core::Relation': 46,
 'Dublin Core::Rights': 47,
 'Dublin Core::Source': 48,
 'Dublin Core::Subject': 49,
 'Dublin Core::Title': 50,
 'Dublin Core::Type': 51,
 'IIIF File Metadata::JSON Data': 53,
 'IIIF File Metadata::Original @id': 52,
 'IIIF Item Metadata::Display as IIIF?': 54,
 'IIIF Item Metadata::JSON Data': 56,
 'IIIF Item Metadata::Original @id': 55,
 'IIIF Item Metadata::UUID': 64}

In [6]:
endpoint = "https://api.harvardartmuseums.org/"
apikey = "b0cde630-ce66-11e8-951c-b3d75228cc98"

In [7]:
Q = {"apikey":apikey,"q":"objectnumber:BR65.56"}
R = requests.get(endpoint+"object",params=Q)

In [8]:
def get_record(objectnumber):
    Q = {"apikey":apikey,"q":"objectnumber:{}".format(objectnumber)}
    R = requests.get(endpoint+"object",params=Q)
    results = R.json()
    if results['info']['totalrecords'] > 1:
        print("more than one result for{}, taking first anyways".format(objectnumber))
    record = results['records'][0]
    return record

In [9]:
def get_objects(endpoint,apikey,**kwargs):
    Q = kwargs
    Q['apikey'] = apikey
    Q['page'] = 1
    R = requests.get(endpoint+"object",params=Q)
    results = R.json()
    print("got results from {}".format(R.url))
    Q['page'] += 1
    while Q['page'] <= results['info']['pages']:
        newResults = requests.get(endpoint+"object",params=Q)
        results['records'].extend(newResults.json()['records'])
        Q['page'] += 1
        print("got results from {}".format(newResults.url))
    return results

In [10]:
objs = get_objects(endpoint,apikey,exhibition=5295)

got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=1
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=2
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=3
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=4
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=5
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=6
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=7
got results from https://api.harvardartmuseums.org/object?exhibition=5295&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=8


In [147]:
len(objs['records'])

75

In [12]:
def get_manifest(record):
    for see in record['seeAlso']:
        if see['type'] == "IIIF Manifest":
            manifest = requests.get(see['id']).json()
            return manifest

In [13]:
def get_canvas(manifest):
    try:
        return manifest['sequences'][0]['canvases'][0]
    except KeyError:
        print(manifest.keys())
        return None

In [112]:
R = requests.get("https://ids.lib.harvard.edu/ids/iiif/12463467/full/!170,170/0/native.jpg")

In [14]:
def create_item_from_obj_number(objectnumber,site):
    record = get_record(objectnumber)
    manifest = get_manifest(record)
    canvas = get_canvas(manifest)
    title = record['title']
    source = "<a href='{url}'>{creditline}</a>".format(**record)
    elements = []
    elements.append(site.item_element(lookup['Dublin Core::Title'],title))
    elements.append(site.item_element(lookup['Dublin Core::Source'],source,html=True))
    elements.append(site.item_element(lookup['IIIF Item Metadata::JSON Data'],json.dumps(canvas)))
    item = site.make_item(elements,public=True,featured=True)
    added_item = site.post_item(item)
    print("added item, ID {}".format(added_item['id']))
    thumbnail = requests.get(manifest['thumbnail']['@id']).content
    files = {
        "file":(manifest['thumbnail']['@id'], thumbnail, "application/octet-stream"),
        "data":(None, json.dumps({"item":{"id":added_item['id']}}))
    }
    R = requests.post("https://cb51.omeka.fas.harvard.edu/api/files", params={"key":"630d3045363b980294b3fed669f37c67d785bb9b"}, files=files)
    return R

In [149]:
for record in objs['records']:
    manifest = get_manifest(record)
    canvas = get_canvas(manifest)
    title = record['title']
    source = "<a href='{url}'>{creditline}</a>".format(**record)
    description = record['description']
    elements = []
    elements.append(site.item_element(lookup['Dublin Core::Title'],title))
    elements.append(site.item_element(lookup['Dublin Core::Source'],source,html=True))
    elements.append(site.item_element(lookup['Dublin Core::Description'],description))
    elements.append(site.item_element(lookup['IIIF Item Metadata::JSON Data'],json.dumps(canvas)))
    item = site.make_item(elements)
    added_item = site.post_item(item)
    print("added item, id {}".format(added_item['id']))
    thumbnail = requests.get(manifest['thumbnail']['@id']).content
    files = {
        "file":(manifest['thumbnail']['@id'], thumbnail, "application/octet-stream"),
        "data":(None, json.dumps({'item':{'id':added_item['id']}}))
    }
    R = requests.post("https://cb51.omeka.fas.harvard.edu/api/files", params={"key":"630d3045363b980294b3fed669f37c67d785bb9b"}, files=files)
    print("added file to item")

added item, id 74
added file to item
added item, id 75
added file to item
added item, id 76
added file to item
added item, id 77
added file to item
added item, id 78
added file to item
added item, id 79
added file to item
added item, id 80
added file to item
added item, id 81
added file to item
added item, id 82
added file to item
added item, id 83
added file to item
added item, id 84
added file to item
added item, id 85
added file to item
added item, id 86
added file to item
added item, id 87
added file to item
added item, id 88
added file to item
added item, id 89
added file to item
added item, id 90
added file to item
added item, id 91
added file to item
added item, id 92
added file to item
added item, id 93
added file to item
added item, id 94
added file to item
added item, id 95
added file to item
added item, id 96
added file to item
added item, id 97
added file to item
added item, id 98
added file to item
added item, id 99
added file to item
added item, id 100
added file to item


In [150]:
import pandas as pd

In [152]:
df = pd.DataFrame(objs['records'])

In [158]:
df.columns

Index(['accessionmethod', 'accessionyear', 'accesslevel', 'century',
       'classification', 'classificationid', 'colorcount', 'colors',
       'commentary', 'contact', 'contextualtextcount', 'copyright',
       'creditline', 'culture', 'datebegin', 'dated', 'dateend',
       'dateoffirstpageview', 'dateoflastpageview', 'department',
       'description', 'dimensions', 'division', 'edition', 'exhibitioncount',
       'groupcount', 'id', 'imagecount', 'imagepermissionlevel', 'images',
       'labeltext', 'lastupdate', 'markscount', 'mediacount', 'medium',
       'objectid', 'objectnumber', 'people', 'peoplecount', 'period',
       'periodid', 'primaryimageurl', 'provenance', 'publicationcount', 'rank',
       'relatedcount', 'seeAlso', 'signed', 'standardreferencenumber', 'state',
       'style', 'technique', 'techniqueid', 'title', 'titlescount',
       'totalpageviews', 'totaluniquepageviews', 'url', 'verificationlevel',
       'verificationleveldescription', 'worktypes'],
      dtyp

In [161]:
df = df[['accessionmethod', 'accessionyear', 'accesslevel', 'century',
       'classification', 'classificationid', 'colorcount',
       'commentary', 'contact', 'contextualtextcount', 'copyright',
       'creditline', 'culture', 'datebegin', 'dated', 'dateend',
       'dateoffirstpageview', 'dateoflastpageview', 'department',
       'description', 'dimensions', 'division', 'edition', 'exhibitioncount',
       'groupcount', 'id', 'imagecount', 'imagepermissionlevel',
       'labeltext', 'lastupdate', 'markscount', 'mediacount', 'medium',
       'objectid', 'objectnumber', 'people', 'peoplecount', 'period',
       'periodid', 'primaryimageurl', 'provenance', 'publicationcount', 'rank',
       'relatedcount', 'signed', 'standardreferencenumber', 'state',
       'style', 'technique', 'techniqueid', 'title', 'titlescount',
       'totalpageviews', 'totaluniquepageviews', 'url', 'verificationlevel',
       'verificationleveldescription']]

In [162]:
df.to_csv("animal_vessels.csv")

In [129]:
response = create_item_from_obj_number("1970.44",site)

added item, ID 31


In [16]:
import json
objects = [
    "1975.41.138"
]
for obj in objects:
    create_item_from_obj_number(obj,site)

added item, ID 150


In [126]:
type(response)

requests.models.Response

In [127]:
response.text

'{"id":23,"url":"https:\\/\\/cb51.omeka.fas.harvard.edu\\/api\\/files\\/23","file_urls":{"original":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Foriginal%2F477e534fd94bf8d9cb0bb2771db72384.jpg","fullsize":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Ffullsize%2F477e534fd94bf8d9cb0bb2771db72384.jpg","thumbnail":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Fthumbnails%2F477e534fd94bf8d9cb0bb2771db72384.jpg","square_thumbnail":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Fsquare_thumbnails%2F477e534fd94bf8d9cb0bb2771db72384.jpg"},"added":"2018-10-17T15:39:10+00:00","modified":"2018-10-17T15:39:11+00:00","filename":"477e534fd94bf8d9cb0bb2771db72384.jpg","authentication":"10399f3f747a4849c4a2b15acb48a598","has_derivative_image":true,"mime_type":"image\\/jpeg","order":null,"original_filename":"native.jpg","size":7944,"stored":true,"type_os":"JPEG image data, JFIF standard 1.01, aspect ratio, density 1x1, segment length 16, baseline, precision 8, 116x170, frames 3","metad

In [81]:
item = {"item":{"id":"24"}}
files = {
    "file":("1200px-US_Capitol_west_side.jpg", open("/Users/jguillette/Pictures/1200px-US_Capitol_west_side.jpg","rb"),"application/octet-stream"),
    "data":(None, json.dumps(item), "application/json")
}
R = requests.post("https://cb51.omeka.fas.harvard.edu/api/files", params={"key":"630d3045363b980294b3fed669f37c67d785bb9b"}, files=files, data=old)

In [108]:
item = item_from_obj_number('1970.44')

In [109]:
item

{'element_texts': [{'element': {'id': 50},
   'html': False,
   'text': 'Annunciation'},
  {'element': {'id': 48},
   'html': False,
   'text': "<a href='https://www.harvardartmuseums.org/collections/object/227811'>Harvard Art Museums/Fogg Museum, Anonymous Gift</a>"},
  {'element': {'id': 53},
   'html': False,
   'text': '{"@id": "https://iiif.harvardartmuseums.org/manifests/object/227811/canvas/canvas-12463467", "@type": "sc:Canvas", "height": 1024, "images": [{"@id": "https://iiif.harvardartmuseums.org/manifests/object/227811/annotation/anno-12463467", "@type": "oa:Annotation", "motivation": "sc:painting", "on": "https://iiif.harvardartmuseums.org/manifests/object/227811/canvas/canvas-12463467", "resource": {"@id": "https://ids.lib.harvard.edu/ids/iiif/12463467/full/full/0/native.jpg", "@type": "dctypes:Image", "format": "image/jpeg", "height": 1024, "service": {"@context": "http://iiif.io/api/image/2/context.json", "@id": "https://ids.lib.harvard.edu/ids/iiif/12463467", "profile":

In [90]:
manifest = get_manifest(test)

In [95]:
manifest['sequences'][0]['canvases'][0]

{'@id': 'https://iiif.harvardartmuseums.org/manifests/object/223185/canvas/canvas-20684286',
 '@type': 'sc:Canvas',
 'height': 801,
 'images': [{'@id': 'https://iiif.harvardartmuseums.org/manifests/object/223185/annotation/anno-20684286',
   '@type': 'oa:Annotation',
   'motivation': 'sc:painting',
   'on': 'https://iiif.harvardartmuseums.org/manifests/object/223185/canvas/canvas-20684286',
   'resource': {'@id': 'https://ids.lib.harvard.edu/ids/iiif/20684286/full/full/0/native.jpg',
    '@type': 'dctypes:Image',
    'format': 'image/jpeg',
    'height': 801,
    'service': {'@context': 'http://iiif.io/api/image/2/context.json',
     '@id': 'https://ids.lib.harvard.edu/ids/iiif/20684286',
     'profile': 'http://iiif.io/api/image/2/level2.json'},
    'width': 1024}}],
 'label': 'center\r\n',
 'otherContent': [{'@id': 'https://iiif.harvardartmuseums.org/manifests/object/223185/list/20684286',
   '@type': 'sc:AnnotationList'}],
 'width': 1024}

In [18]:
test = get_record("BR58.212")

In [97]:
test

{'accessionmethod': 'Purchase',
 'accessionyear': 1958,
 'accesslevel': 1,
 'century': '7th century',
 'classification': 'Jewelry',
 'classificationid': 19,
 'colorcount': 0,
 'commentary': None,
 'contact': 'am_europeanamerican@harvard.edu',
 'contextualtextcount': 0,
 'copyright': None,
 'creditline': 'Harvard Art Museums/Busch-Reisinger Museum, Purchase in memory of Eda K. Loeb',
 'culture': 'German',
 'datebegin': 0,
 'dated': '7th century',
 'dateend': 0,
 'dateoffirstpageview': '2011-08-18',
 'dateoflastpageview': '2014-03-01',
 'department': 'Busch-Reisinger Museum',
 'description': None,
 'dimensions': 'irregular: 6.5 cm (2 9/16 in.)',
 'division': 'European and American Art',
 'edition': None,
 'exhibitioncount': 2,
 'groupcount': 0,
 'id': 223185,
 'imagecount': 1,
 'imagepermissionlevel': 0,
 'images': [{'baseimageurl': 'https://nrs.harvard.edu/urn-3:HUAM:VRS64032_dynmc',
   'copyright': 'President and Fellows of Harvard College',
   'displayorder': 1,
   'format': 'image/jp

In [35]:
elements = []
elements.append(site.item_element(lookup['Dublin Core::Title'],test['title']))
elements.append(site.item_element(lookup['Dublin Core::Source'],"<a href='https://www.harvardartmuseums.org/collections/object/{}'>Harvard Art Museums</a>".format(test['objectid']),html=True))
manifest = ""
for see in test['seeAlso']:
    if see['type'] == "IIIF Manifest":
        manifest = requests.get(see['id']).text
elements.append(site.item_element(lookup['IIIF File Metadata::JSON Data'], manifest))
elements

[{'element': {'id': 50}, 'html': False, 'text': 'Pendant of Three Snakes'},
 {'element': {'id': 48},
  'html': True,
  'text': "<a href='https://www.harvardartmuseums.org/collections/object/223185'>Harvard Art Museums</a>"},
 {'element': {'id': 53},
  'html': False,
  'text': '{"@context": "http://iiif.io/api/presentation/2/context.json", "@id": "https://iiif.harvardartmuseums.org/manifests/object/223185", "@type": "sc:Manifest", "attribution": "Harvard Art Museums", "description": "", "label": "Pendant of Three Snakes", "logo": {"@id": "https://ids.lib.harvard.edu/ids/iiif/437958013/full/!800,800/0/native.jpg", "service": {"@context": "http://iiif.io/api/image/2/context.json", "@id": "https://ids.lib.harvard.edu/ids/iiif/437958013", "profile": "http://iiif.io/api/image/2/level2.json"}}, "metadata": [{"label": "Date", "value": "7th century"}, {"label": "Classification", "value": "Jewelry"}, {"label": "Credit Line", "value": "Harvard Art Museums/Busch-Reisinger Museum, Purchase in memor

In [36]:
item = site.make_item(elements)

In [37]:
site.post_item(item)

{'added': '2018-10-15T21:54:38+00:00',
 'collection': None,
 'element_texts': [{'element': {'id': 50,
    'name': 'Title',
    'resource': 'elements',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/elements/50'},
   'element_set': {'id': 1,
    'name': 'Dublin Core',
    'resource': 'element_sets',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/element_sets/1'},
   'html': False,
   'text': 'Pendant of Three Snakes'},
  {'element': {'id': 48,
    'name': 'Source',
    'resource': 'elements',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/elements/48'},
   'element_set': {'id': 1,
    'name': 'Dublin Core',
    'resource': 'element_sets',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/element_sets/1'},
   'html': True,
   'text': "<a href='https://www.harvardartmuseums.org/collections/object/223185'>Harvard Art Museums</a>"},
  {'element': {'id': 53,
    'name': 'JSON Data',
    'resource': 'elements',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/elements/53'},
   

In [40]:
R = requests.get("https://cb51.omeka.fas.harvard.edu/api/items/13")
old = R.json()

In [41]:
old.pop('id')

13

In [42]:
site.post_item(old)

{'added': '2018-10-16T15:59:52+00:00',
 'collection': None,
 'element_texts': [{'element': {'id': 50,
    'name': 'Title',
    'resource': 'elements',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/elements/50'},
   'element_set': {'id': 1,
    'name': 'Dublin Core',
    'resource': 'element_sets',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/element_sets/1'},
   'html': False,
   'text': 'King John of France'},
  {'element': {'id': 48,
    'name': 'Source',
    'resource': 'elements',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/elements/48'},
   'element_set': {'id': 1,
    'name': 'Dublin Core',
    'resource': 'element_sets',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/element_sets/1'},
   'html': False,
   'text': 'https://iiif.harvardartmuseums.org/manifests/object/229262'},
  {'element': {'id': 55,
    'name': 'Original @id',
    'resource': 'elements',
    'url': 'https://cb51.omeka.fas.harvard.edu/api/elements/55'},
   'element_set': {'id': 5,
    'name'

In [64]:
R = requests.get("https://cb51.omeka.fas.harvard.edu/api/items/24?key=630d3045363b980294b3fed669f37c67d785bb9b")
old = R.json()

In [76]:
import json

In [82]:
R.text

'{"id":18,"url":"https:\\/\\/cb51.omeka.fas.harvard.edu\\/api\\/files\\/18","file_urls":{"original":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Foriginal%2Fb8fe36baa1672354c3aedc8c68417f6e.jpg","fullsize":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Ffullsize%2Fb8fe36baa1672354c3aedc8c68417f6e.jpg","thumbnail":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Fthumbnails%2Fb8fe36baa1672354c3aedc8c68417f6e.jpg","square_thumbnail":"http:\\/\\/s3.amazonaws.com\\/oaas-files%2Fcb51%2Fsquare_thumbnails%2Fb8fe36baa1672354c3aedc8c68417f6e.jpg"},"added":"2018-10-16T16:44:13+00:00","modified":"2018-10-16T16:44:14+00:00","filename":"b8fe36baa1672354c3aedc8c68417f6e.jpg","authentication":"c53a14995e9de8886daf2535b8e7679d","has_derivative_image":true,"mime_type":"image\\/jpeg","order":null,"original_filename":"1200px-US_Capitol_west_side.jpg","size":291831,"stored":true,"type_os":"JPEG image data, JFIF standard 1.02, resolution (DPI), density 600x600, segment length 16, baseline, precisi

In [68]:
R.url

'https://cb51.omeka.fas.harvard.edu/api/files?key=630d3045363b980294b3fed669f37c67d785bb9b'

In [ ]:
payload = {"param_1": "value_1", "param_2": "value_2"}
files = {
     'json': (None, json.dumps(payload), 'application/json'),
     'file': (os.path.basename(file), open(file, 'rb'), 'application/octet-stream')
}

r = requests.post(url, files=files)

In [5]:
R.json()

{'info': {'page': 1,
  'pages': 1,
  'totalrecords': 1,
  'totalrecordsperquery': 10},
 'records': [{'accessionmethod': 'Gift',
   'accessionyear': 1965,
   'accesslevel': 1,
   'century': '15th century',
   'classification': 'Vessels',
   'classificationid': 57,
   'colorcount': 0,
   'commentary': None,
   'contact': 'am_europeanamerican@harvard.edu',
   'contextualtextcount': 0,
   'copyright': None,
   'creditline': 'Harvard Art Museums/Busch-Reisinger Museum, Gift of Mrs. George S. Selfridge',
   'culture': 'German',
   'datebegin': 0,
   'dated': '15th century',
   'dateend': 0,
   'dateoffirstpageview': '2010-03-29',
   'dateoflastpageview': '2018-10-06',
   'department': 'Busch-Reisinger Museum',
   'description': None,
   'dimensions': '26.67 cm (10 1/2 in.)',
   'division': 'European and American Art',
   'edition': None,
   'exhibitioncount': 2,
   'groupcount': 0,
   'id': 222649,
   'imagecount': 1,
   'imagepermissionlevel': 0,
   'images': [{'baseimageurl': 'https://nrs.

- Get base info, including title and object id
- get manifest url, manifest in text
- create item with other codebase
- add item